In [1]:
import datetime
import time
from collections import defaultdict

import psycopg2
import pandas as pd
from cassandra.cluster import Cluster
from solutions import RelationalSolution, ColumnSolution
from solutionbase import generate_entries, Solution, DateInterval

In [2]:
generated_data = generate_entries()

In [3]:
relational_solution = RelationalSolution(
    psycopg2.connect(host='localhost', dbname='store', user='p_user', password='p_password')
)

cluster = Cluster()
column_solution = ColumnSolution(cluster.connect('store'))
all_solutions: list[tuple[str, Solution]] = [
    ('реляційна', relational_solution),
    ('стовпчикова', column_solution),
]

In [4]:
def get_test_cases(solution: Solution):
    date_interval = DateInterval(
        start=datetime.datetime(2022, 1, 1),
        end=datetime.datetime(2023, 1, 1),
    )
    target_product = 'Product_0'
    target_shop = 'Shop_1'
    return [
        ('Порахувати кількість проданового товару', lambda: solution.get_total_amount()),
        ('Порахувати вартість проданого товару', lambda: solution.get_total_price()),
        (
            'Порахувати вартість проданого товару за період',
            lambda: solution.get_total_price_for_date_interval(date_interval)
        ),
        (
            'Порахувати скільки було придбано товару А в мазазині В за період С',
            lambda: solution.get_amount_at_shop(target_product, target_shop, date_interval)
        ),
        (
            'Порахувати скільки було придбано товару А в усіх магазинах за період С',
            lambda: solution.get_amount(target_product, date_interval)
        ),
        (
            'Порахувати сумарну виручку магазинів за період С',
            lambda: solution.get_total_price_by_shop(date_interval)
        ),
        (
            'Вивести топ 10 купівель товарів по два за період С',
            lambda: solution.get_top_products_by_2(date_interval, limit=10),
        ),
        (
            'Вивести топ 10 купівель товарів по три за період С',
            lambda: solution.get_top_products_by_3(date_interval, limit=10),
        ),
        (
            'Вивести топ 10 купівель товарів по чотири за період С',
            lambda: solution.get_top_products_by_4(date_interval, limit=10),
        ),
    ]

## Сгенеруємо тестові дані

In [5]:
for _, solution in all_solutions:
    solution.populate(generated_data)

# Перевіримо коректність роботи запитів 

In [6]:
results_table = defaultdict(defaultdict)
for solution_name, solution in all_solutions:
    for test_name, test_func in get_test_cases(solution):
        results_table[solution_name][test_name] = test_func()

In [7]:
pd.DataFrame(results_table)

,реляційна,стовпчикова
Порахувати кількість проданового товару,8101.86419,8101.864258
Порахувати вартість проданого товару,5435003,5435003
Порахувати вартість проданого товару за період,906576,906576
Порахувати скільки було придбано товару А в мазазині В за період С,13.910051,13.91005
Порахувати скільки було придбано товару А в усіх магазинах за період С,131.126897,131.126892
Порахувати сумарну виручку магазинів за період С,"[(Shop_0, 105708), (Shop_1, 77935), (Shop_2, 9...","[(Shop_5, 85650), (Shop_0, 105708), (Shop_4, 1..."
Вивести топ 10 купівель товарів по два за період С,"[(Product_4, Product_7, 72), (Product_1, Produ...","[(Product_4, Product_7, 72), (Product_6, Produ..."
Вивести топ 10 купівель товарів по три за період С,"[(Product_5, Product_6, Product_8, 57), (Produ...","[(Product_5, Product_6, Product_8, 57), (Produ..."
Вивести топ 10 купівель товарів по чотири за період С,"[(Product_1, Product_6, Product_7, Product_8, ...","[(Product_1, Product_6, Product_7, Product_8, ..."


## Виміряємо час роботи запитів

In [8]:
timing_table = defaultdict(defaultdict)
for solution_name, solution in all_solutions:
    for test_name, test_func in get_test_cases(solution):
        start_time = time.time()
        test_func()
        end_time = time.time()
        timing_table[f"{solution_name} (мс)"][test_name] = (end_time - start_time) * 1000

In [9]:
pd.DataFrame(timing_table)

,реляційна (мс),стовпчикова (мс)
Порахувати кількість проданового товару,1.411200,6.703854
Порахувати вартість проданого товару,0.837803,6.887913
Порахувати вартість проданого товару за період,0.941992,4.628897
Порахувати скільки було придбано товару А в мазазині В за період С,0.909090,1.914024
Порахувати скільки було придбано товару А в усіх магазинах за період С,0.938177,9.675980
Порахувати сумарну виручку магазинів за період С,0.968933,5.684853
Вивести топ 10 купівель товарів по два за період С,3.006935,5.679131
Вивести топ 10 купівель товарів по три за період С,10.712147,8.050919
Вивести топ 10 купівель товарів по чотири за період С,24.456978,11.507034
